In [8]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

from pathlib import Path

model = timm.create_model('mobilenetv3_large_100.ra_in1k', pretrained=True, num_classes = 2)
model.requires_grad_(False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torch/storage.py:899: UserWarning: TypedStorage is d

### Dataloader

In [9]:
train_transforms = transforms([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomErasing(),
    transforms.RandAugment(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transforms = transforms([
    transforms.RandomRotation(5),
    transforms.RandomErasing(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

TypeError: __init__() missing 1 required positional argument: 'std'

In [ ]:
data_path = str(Path('/home/mosminin/dev/yolo4scaled/darknet/parser/datasets/COCO2017/person_class'))

train_dataset = ImageFolder(root=data_path+'/train', transform=train_transforms)
val_dataset = ImageFolder(root=data_path+'/valid', transform=valid_transforms)

### Train

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

### test